In [1]:
import pymysql
import colorExtract

In [19]:
def insertColor(color, id):
    conn = pymysql.connect(host='127.0.0.1', user='ssafy', password='ssafy', db='kkalong', charset='utf8')
    cur = conn.cursor()
    sql = "update clothing set color = %s where clothing_id = %s"
    cur.execute(sql, (color, id));
    conn.commit()
    conn.close()

In [18]:
conn = pymysql.connect(host='127.0.0.1', user='ssafy', password='ssafy', db='kkalong', charset='utf8')
cur = conn.cursor()
cur.execute("select * from clothing");

bottom_blue = ['민트', '스카이 블루', '네온 블루', '라벤더', '연청']
bottom_navy = ['네이비', '보라색', '데님', '파란색', '중청', '진청']
bottom_white = ["화이트", "아이보리", "샌드", "베이지", "페일 핑크", '라이트 그레이', '회색']
bottom_khaki = ['네온 그린', '라이트 그린', '녹색', '올리브 그린', '카키', '다크 그린', '카키 베이지',
                '라이트 옐로우', '노란색', '머스타드', '카멜', '라이트 오렌지', '네온 오렌지', '주황색', '레드 브라운']
bottom_wine = ["딥레드", "빨간색", "자주", "버건디", "갈색", "로즈 골드", "라즈베리", "네온 핑크", "분홍색", "라이트 핑크", "피치", "오렌지 핑크"]
bottom_black = ['블랙', '흑청', '다크 그레이']

color_top = ["white", "red", "pink", "orange", "yellow", "greene", "blue", "navy", "black", "gray"]
color_bottom = ["white", "blue", "wine", "black", "khaki", "navy"]

top_bottom = [
                [top_white, bottom_blue], [top_white, bottom_navy], [top_white, bottom_white],
                [top_white, bottom_khaki], [top_white, bottom_wine], [top_white, bottom_white],
                [top_red, bottom_wine], [top_pink, bottom_blue], [top_pink, bottom_navy],
                [top_pink, bottom_white], [top_pink, bottom_black], [top_orange, bottom_navy],
                [top_yellow, bottom_blue], [top_yellow, bottom_navy], [top_yellow, bottom_white],
                [top_yellow, bottom_wine], [top_green, bottom_navy], [top_green, bottom_black],
                [top_blue, bottom_white], [top_blue, bottom_wine], [top_blue, bottom_black],
                [top_navy, bottom_blue], [top_navy, bottom_navy], [top_navy, bottom_white],
                [top_navy, bottom_wine], [top_black, bottom_navy], [top_black, bottom_white],
                [top_black, bottom_black], [top_gray, bottom_black], [top_gray, bottom_navy]
            ]

1
(218, 211, 200)
라이트그레이
2
(72, 66, 60)
카키
3
(169, 152, 113)
회색
4
(112, 79, 55)
카키
5
(142, 125, 87)
올리브 그린
6
(224, 224, 224)
라이트그레이
7
(89, 89, 78)
다크 그레이
8
(227, 222, 214)
라이트그레이
9
(131, 143, 156)
회색
10
(64, 62, 69)
중청
11
(82, 49, 26)
갈색
12
(200, 189, 190)
라이트그레이
13
(44, 42, 38)
흑청
14
(160, 174, 201)
회색
15
(218, 219, 208)
라이트그레이
16
(231, 231, 231)
라이트그레이
17
(80, 84, 59)
카키
18
(164, 124, 84)
올리브 그린
19
(168, 87, 69)
로즈골드
20
(148, 111, 81)
올리브 그린
21
(64, 67, 51)
카키
22
(53, 53, 61)
진청
23
(130, 135, 150)
회색
24
(127, 95, 67)
카키
25
(231, 221, 205)
라이트그레이
26
(203, 197, 184)
샌드
27
(142, 107, 73)
올리브 그린
28
(24, 29, 48)
진청
29
(200, 200, 209)
라이트그레이
30
(25, 24, 23)
흑청
31
(40, 41, 51)
진청
32
(32, 37, 48)
진청
33
(106, 120, 90)
올리브 그린
34
(225, 225, 219)
라이트그레이
35
(195, 187, 184)
샌드
36
(99, 106, 114)
다크 그레이
37
(80, 80, 77)
다크 그레이
38
(106, 73, 43)
카키
39
(55, 61, 47)
진청
40
(69, 65, 65)
카키
41
(52, 48, 28)
흑청
42
(104, 93, 88)
다크 그레이
43
(25, 23, 30)
흑청
44
(35, 35, 35)
흑청
45
(38, 37, 37)
흑청
46
(58, 58, 58)
진청

In [31]:
a = []
for i in top_bottom:
    if i.__contains__(top_white):
        a.append(i[1])
print(a)

[['민트', '스카이 블루', '네온 블루', '라벤더', '연청'], ['네이비', '보라색', '데님', '파란색', '중청', '진청'], ['화이트', '아이보리', '샌드', '베이지', '페일 핑크', '라이트 그레이', '회색'], ['네온 그린', '라이트 그린', '녹색', '올리브 그린', '카키', '다크 그린', '카키 베이지', '라이트 옐로우', '노란색', '머스타드', '카멜', '라이트 오렌지', '네온 오렌지', '주황색', '레드 브라운'], ['딥레드', '빨간색', '자주', '버건디', '갈색', '로즈 골드', '라즈베리', '네온 핑크', '분홍색', '라이트 핑크', '피치', '오렌지 핑크'], ['화이트', '아이보리', '샌드', '베이지', '페일 핑크', '라이트 그레이', '회색']]


In [29]:
cody_sub = {1: 1, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0}
print(cody_sub[1])

1
